# Scraping Thơ Đỗ Phủ trên thivien.net
- lấy url của các bài thơ
- mở url lấy nội dung thơ (chữ Hán, phiên âm Hán Việt, phiên dịch)
*trang block scrap nên phải dùng header 

In [4]:
import requests
from bs4 import BeautifulSoup
import random
import csv
import re
import pandas as pd
import requests

from docx import Document
import time



Phần scrap url

In [20]:


user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
]

# URL của trang web cần scrap
url = "https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/author-Se3x2jWTezvoDpkpSU9qjg"

try:
    # Tải nội dung của trang web
    print("Đang tải nội dung trang web...")
    response = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
    response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt
    print("Nội dung trang web đã được tải thành công.")

    # Phân tích HTML content
    print("Phân tích HTML content...")
    soup = BeautifulSoup(response.content, "html.parser")

    # Trích xuất các URL từ trang web chứa từ "poem" nhưng không lấy từ "searchpoem"
    print("Trích xuất các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem'...")
    urls = [a["href"] for a in soup.find_all("a", href=True) if "poem" in a["href"] and "searchpoem" not in a["href"] and "addpoem" not in a["href"] and "m-poems" not in a["href"]]


    # Lưu các URL vào tệp CSV
    file_path = "D:/Dophu-url-poem.csv"
    with open(file_path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["URL"])

        for url in urls:
            writer.writerow([url])

    print(f"Các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem' đã được lưu vào {file_path}")

except requests.exceptions.RequestException as e:
    print("Lỗi khi tải URL:", e)
except Exception as e:
    print("Đã xảy ra lỗi:", e)


Đang tải nội dung trang web...
Nội dung trang web đã được tải thành công.
Phân tích HTML content...
Trích xuất các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem'...
Các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem' đã được lưu vào D:/Dophu-url-poem.csv


In [5]:
# Đọc file csv chứa các URL
df = pd.read_csv("D:/Dophu-url-poem.csv")
# Khởi tạo danh sách để lưu trữ dữ liệu
data = []
# Hàm để thêm phần đầu của URL
def add_prefix(url):
    return "https://www.thivien.net" + url

# Thêm phần đầu vào từng URL
df['Full_URL'] = df['URL'].apply(add_prefix)

# Hàm để trích xuất dữ liệu từ URL và lưu vào file .docx
def extract_poem_data(url):
    user_agents_list = [
        'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    ]
    HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
    try:
        # Delay 15 giây trước khi tải nội dung trang web
        random_sleep_time = random.randint(8, 15)
        time.sleep(random_sleep_time)
       
        
        # Tải nội dung của trang web
        print("Đang tải nội dung trang web:", url)
        #response = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt
   
        # Tạo biến đếm cho số lần đã mở link
        link_count = 0
        
        # Phân tích HTML content
        soup = BeautifulSoup(response.content, "html.parser")

        # Trích xuất tiêu đề
        title = soup.title.text.strip()
        print("Tiêu đề:", title)

        # Trích xuất phần văn bản chữ Hán và chữ Việt
        han_viet_text = ""
        for p_tag in soup.find_all('p', class_='HanChinese transcriptable'):
            han_viet_text += p_tag.text.strip() + "\n"

        # Tìm thẻ 
        
            
        
        h4_elements = soup.find_all("h4", string=True)
        for h4_element in h4_elements:
            if h4_element.text.strip() != "Đăng nhập" and h4_element.text.strip() != "Dịch nghĩa":
                #print("Tiêu đề:", h4_element.text.strip())
                p_tag = h4_element.find_next_sibling("p")
                
                if p_tag:
                    #print("Nội dung p:", p_tag.text.strip()) 
                    description_content =  p_tag.text.strip()
                    
        # Tìm tất cả các thẻ span có class là "gray"
        gray_spans = soup.find_all("span", class_="gray")

        # Lưu nội dung của từng thẻ span gray vào một list
        translations = []
        for gray_span in gray_spans:
            em_tag = gray_span.find("em")
            if em_tag:
                content_text = em_tag.text.strip()
                # Thêm kí tự xuống dòng vào cuối mỗi nội dung
                
                translations.append(content_text)
               
                
       
        # Tạo dữ liệu mới
        poem_data = {
            'Tiêu đề': title,
            'Chữ Hán': han_viet_text,
            'Phiên âm': description_content,
            'Phiên dịch': '\n'.join(translations)
        }
        
        # Thêm dữ liệu vào danh sách
        data.append(poem_data)
        df_poems = pd.DataFrame(data)
        df_poems.to_csv("D:/dophu_poem.csv", index=False, encoding='utf-8-sig')
        print("đÃ CẬP NHẬT FILE")
         # Tăng biến đếm sau mỗi lần mở link
        link_count += 1
        # Kiểm tra nếu biến đếm chia hết cho 5 thì nghỉ ngơi 10 giây
        if link_count % 5 == 0:
            print(f"Đang chờ ngẫu nhiên từ {random_sleep_time} giây...")
           
            random_sleep_time = random.randint(15, 25)
            
           
            time.sleep(random_sleep_time)
        
    except requests.exceptions.RequestException as e:
        print("Lỗi khi tải URL:", e)
    except Exception as e:
        print("Đã xảy ra lỗi:", e)

# Gọi hàm để trích xuất dữ liệu từ từng URL và lưu vào file .docx
for url in df['Full_URL']:
    extract_poem_data(url)
# Tạo DataFrame từ danh sách dữ liệu
df_poems = pd.DataFrame(data)

# Lưu DataFrame vào tệp CSV
df_poems.to_csv("D:/dophu_poem.csv", index=False, encoding='utf-8-sig')
print("Đã lưu dữ liệu vào file D:/dophu_poem.csv")


Đang tải nội dung trang web: https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/Thu-h%E1%BB%A9ng-k%E1%BB%B3-1/poem-d-FDjEupVCgpsc1eaacNvg
Tiêu đề: Bài thơ: Thu hứng kỳ 1 - 秋興其一 (Đỗ Phủ - 杜甫)
đÃ CẬP NHẬT FILE
Đang tải nội dung trang web: https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/%C4%90%C4%83ng-cao/poem-h3EvqeQvlQUxq2sceUWWYQ
Tiêu đề: Bài thơ: Đăng cao - 登高 (Đỗ Phủ - 杜甫)
đÃ CẬP NHẬT FILE
Đang tải nội dung trang web: https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/Nguy%E1%BB%87t-d%E1%BA%A1/poem-T7ykhmCIqNvTZ_L2JmhFFw
Tiêu đề: Bài thơ: Nguyệt dạ - 月夜 (Đỗ Phủ - 杜甫)
đÃ CẬP NHẬT FILE
Đang tải nội dung trang web: https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/Mao-%E1%BB%91c-v%E1%BB%8B-thu-phong-s%E1%BB%9F-ph%C3%A1-ca/poem-j4UAOMEqMgeNDayunsRjYA
Tiêu đề: Bài thơ: Mao ốc vị thu phong sở phá ca - 茅屋為秋風所破歌 (Đỗ Phủ - 杜甫)
đÃ CẬP NHẬT FILE
Đang tải nội dung trang web: https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/Xu%C3%A2n-v%E1%BB%8Dng/poem-D1C_2BVK7I--Zczc0EEJ-A
Tiêu đề: Bà

# Phần Test khi chỉ lấy 1 bài thơ

In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Khởi tạo DataFrame để lưu trữ dữ liệu
data = {'Tiêu đề': [], 'Nội dung p': []}

df = pd.DataFrame(data)
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

# URL của trang web cần lấy nội dung
url = "https://www.thivien.net/%C4%90%E1%BB%97-Ph%E1%BB%A7/%E1%BA%A8m-trung-b%C3%A1t-ti%C3%AAn-ca/poem-AmFNdvB_42_rvfX8w3XmLA"

try:
    # Tải nội dung của trang web
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt

    # Phân tích HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Trích xuất tiêu đề
    title = soup.title.text.strip()
    print("Tiêu đề:", title)
    
    # Tìm thẻ h4 và in các đoạn văn bản <p> ngay sau mỗi thẻ h4
    h4_elements = soup.find_all("h4", string=True)
    for h4_element in h4_elements:
        if h4_element.text.strip() != "Đăng nhập" and h4_element.text.strip() != "Dịch nghĩa":
            print("Tiêu đề:", h4_element.text.strip())
            p_tag = h4_element.find_next_sibling("p")
            
            if p_tag:
                print( p_tag.text.strip())
           

except requests.exceptions.RequestException as e:
    print("Lỗi khi tải URL:", e)
except Exception as e:
    print("Đã xảy ra lỗi:", e)


Tiêu đề: Bài thơ: Ẩm trung bát tiên ca - 飲中八仙歌 (Đỗ Phủ - 杜甫)
Tiêu đề: Ẩm trung bát tiên ca
Nội dung p: Tri Chương kỵ mã tự thừa thuyền,Nhãn hoa lạc tỉnh thuỷ để miên.Nhữ Dương tam đấu thuỷ triều thiên.Đạo phùng khúc xa khẩu lưu diên.Hận bất di phong hướng Tửu Tuyền.Tả tướng nhật hứng phí vạn tiền,Ẩm như trường kình hấp bách xuyên.Hàm bôi lạc thánh xưng tị hiền.Tông Chi tiêu sái mỹ thiếu niên,Cử trường bạch nhãn vọng thanh thiên,Hiệu như ngọc thụ lâm phong tiền.Tô Tấn trường trai tú Phật tiền,Tuý trung vãng vãng ái đào thiền.Lý Bạch nhất đấu thi bách thiên,Trường An thị thượng tửu gia miên.Thiên tử hô lai bất thướng thuyền.Tự xưng thần thị tửu trung tiên.Trương Húc tam bôi thảo thánh truyền,Thoát mạo lộ đỉnh vương công tiền.Huy hào lạc chỉ như vân yên.Tiêu Toại ngũ đẩu phương trác nhiên,Cao đàm hùng biện kinh tứ diên.
